# Creating randomized schedule data in a CSV file 

In [9]:
import random
import csv
import os

# --- PARAMETRY ---
days = 30
shifts = 3
employees = 28  # liczba pracowników

folder = "data_csv"
os.makedirs(folder, exist_ok=True)
filename = os.path.join(folder, f"grafik_{days}d_{shifts}s_{employees}emp.csv")

# --- GENEROWANIE CSV ---
with open(filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # --- 1. Nagłówki wymagań ---
    req_headers = []
    req_values = []

    for day in range(1, days + 1):
        for shift in range(1, shifts + 1):
            req_headers.append(f"req_{day}d_{shift}s")
            min_required = max(1, employees // 2)
            if employees==1:
                employees=2
            max_required = employees - 1
            req_values.append(random.randint(min_required, max_required))

    writer.writerow(req_headers)
    writer.writerow(req_values)

    # --- 2. Nagłówki preferencji ---
    pref_headers = [f"pref_{day}d_{shift}s" 
                    for day in range(1, days + 1) 
                    for shift in range(1, shifts + 1)]
    writer.writerow(pref_headers)

    # --- 3. Preferencje pracowników (każda linia = jeden pracownik) ---
    for emp in range(1, employees + 1):
        row = [random.randint(0, 1) for _ in pref_headers]
        writer.writerow(row)

print("CSV wygenerowano:", filename)


CSV wygenerowano: data_csv\grafik_30d_3s_24emp.csv


# Neural Network 

In [9]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
import tensorflow as tf

epoch = 1
file = "genetic_Result"

path = f"example_data_csv/{file}.csv"
data = pd.read_csv(path)

# Kolumny wejściowe (preferencje i wymagania)
X_cols = [
    "day","shift","preference","requirements","singleWorkerFitness"
]

Y_cols = [
    "assigned"
]

# Przygotowanie danych
X = data[X_cols].values
Y = data[Y_cols].values

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Trening modelu
model.fit(X, Y, epochs=epoch, batch_size=32, validation_split=0.2, callbacks=[early_stop])

pred = model.predict(X[:10])
pred_bin = (pred > 0.5).astype(int)

print("Predykcja (pierwsze 10):")
print(np.hstack([pred_bin, Y[:10]]))

# y_pred_binary = (prediction > 0.5).astype(int)

# print("Binarna prognoza (0 = nie pracuje, 1 = pracuje):")
# print(y_pred_binary[0].reshape(3, 3))

# print("Przewidywany harmonogram (3 dni × 3 pracowników):")
# print(test_input)
# print(prediction[0].reshape(3, 3))

# model.save("saved_model/model.keras")

C:\Users\USER098\Documents\GitHub\genekordos\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5366 - loss: 5.5133 - val_accuracy: 0.6414 - val_loss: 0.7503
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Predykcja (pierwsze 10):
[[1 1]
 [1 0]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 0]
 [1 1]
 [1 1]
 [1 1]]
